In [ ]:
# Ml Flow tracking serving
# We need to track each experiment

In [2]:
import mlflow

In [2]:
# set_tracking_uri is a function in the mlflow library that sets the tracking URI for the MLflow tracking server. The tracking URI specifies the location where MLflow logs and stores experiment data, such as metrics, parameters, and artifacts.

# Key Points:
# Purpose: It tells MLflow where to send tracking data (e.g., a local directory, a remote server, or a database).
# Usage: It can be set to a local file path, a remote HTTP/HTTPS server, or other supported backends.
# Effect: Once set, all subsequent MLflow operations (e.g., logging metrics or parameters) will use this URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
mlflow.set_experiment("Check local host connection")

2025/04/24 07:51:01 INFO mlflow.tracking.fluent: Experiment with name 'Check local host connection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/420909352545057414', creation_time=1745499061506, experiment_id='420909352545057414', last_update_time=1745499061506, lifecycle_stage='active', name='Check local host connection', tags={}>

In [5]:
with mlflow.start_run():
    mlflow.log_metric("test", 1)
    mlflow.log_metric("fabio", 2)

🏃 View run bedecked-ram-327 at: http://127.0.0.1:5000/#/experiments/420909352545057414/runs/44c31f9c68164f9699b073adb781d4a5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/420909352545057414


In [3]:
# Experiment tracking example
import mlflow
from mlflow.models import infer_signature
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
# Load the iris dataset
x,y = datasets.load_iris(return_X_y=True)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [7]:
# Define the model hyperparameters
params = {"penalty": "l2", "solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 42}

# Train the model
model = LogisticRegression(**params)
model.fit(x_train, y_train)

/opt/miniconda3/envs/mlops/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=42)

In [8]:
# Prediction
predictions = model.predict(x_test)


In [9]:
# Calculate metrics (accuracy in this case)
accuracy = accuracy_score(y_test, predictions)
accuracy

1.0

In [10]:
# MlFlow experiment tracking
mlflow.set_experiment("Iris_Classification")
with mlflow.start_run():
    # Log parameters
    mlflow.log_params(params)

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag
    mlflow.set_tag("model", "Logistic Regression for iris dataset")

    # Log the model
    signature = infer_signature(x_train, model.predict(x_train))
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="iris_model",
        signature=signature,
        input_example=x_train[:5],  # Optional: Example input for the model
        registered_model_name="Iris_Classifier"  # Optional: Name for the registered model
    )


2025/04/25 08:07:43 INFO mlflow.tracking.fluent: Experiment with name 'Iris_Classification' does not exist. Creating a new experiment.
Successfully registered model 'Iris_Classifier'.
2025/04/25 08:07:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Iris_Classifier, version 1


🏃 View run exultant-hawk-240 at: http://127.0.0.1:5000/#/experiments/120868158865737848/runs/14701c8ea3ab4dc4b3518cf6c7cd3f44
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/120868158865737848


Created version '1' of model 'Iris_Classifier'.


## Validate the model before deployment

In [12]:
import mlflow
from mlflow.models import Model

model_uri = 'runs:/14701c8ea3ab4dc4b3518cf6c7cd3f44/iris_model'
# The model is logged with an input example
pyfunc_model = mlflow.pyfunc.load_model(model_uri)
input_data = pyfunc_model.input_example

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="uv",
)

2025/04/25 08:08:59 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/04/25 08:08:59 INFO mlflow.utils.virtualenv: Creating a new environment in /var/folders/xp/5dmb9r_s3mn9gw4dtc6jz0lr0000gp/T/tmpmuuzlh3s/envs/virtualenv_envs/mlflow-26a1ab1c0576b7d5fb008f0917e25155435e6376 with python version 3.12.9 using uv
Using CPython 3.12.9 interpreter at: /opt/miniconda3/envs/mlops/bin/python3.12
Creating virtual environment at: /var/folders/xp/5dmb9r_s3mn9gw4dtc6jz0lr0000gp/T/tmpmuuzlh3s/envs/virtualenv_envs/mlflow-26a1ab1c0576b7d5fb008f0917e25155435e6376
Activate with: source /var/folders/xp/5dmb9r_s3mn9gw4dtc6jz0lr0000gp/T/tmpmuuzlh3s/envs/virtualenv_envs/mlflow-26a1ab1c0576b7d5fb008f0917e25155435e6376/bin/activate
2025/04/25 08:09:00 INFO mlflow.utils.virtualenv: Installing dependencies
Using Python 3.12.9 environment at: /var/folders/xp/5dmb9r_s3mn9gw4dtc6jz0lr0000gp/T/tmpmuuzlh3s/envs/virtualenv_envs/mlflow-26a1ab1c0576b7d5fb008f0917e25155435e63

{"predictions": [0, 0, 1, 0, 0]}

## Model Registry

The MLFlow Model Registry component is a centralized model store, set of APIs and UI, to collaboratively manage the full lifecycle of MLflow models. It provides a way to manage the model lifecycle, including stages like "Staging", "Production", and "Archived". The Model Registry allows you to version your models, annotate them with metadata, and manage their deployment.

In [13]:
# Create anew MlFlow experiment
mlflow.set_experiment("MLFLOW Quickstart")

# Start an MLFLOW run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the accuracy metrics
    mlflow.log_metric("accuracy", 1.0)

    # Set the tag we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Logistic Regression for iris dataset")

    # infer signature
    signature = infer_signature(x_train, model.predict(x_train))

    # Log the model
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="iris_model",
        signature=signature,
        input_example=x_train[:5],  # Optional: Example input for the model
        # registered_model_name="Iris_Classifier"  # Optional: Name for the registered model
    )


2025/04/25 16:44:45 INFO mlflow.tracking.fluent: Experiment with name 'MLFLOW Quickstart' does not exist. Creating a new experiment.


🏃 View run loud-crab-539 at: http://127.0.0.1:5000/#/experiments/265856417353344936/runs/d411c55d7b224358a5394626ecaea14d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/265856417353344936


In [14]:
# Inferencing from model registry
import mlflow.sklearn
model_name = "Iris_Classifier"
model_version = 2
model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.sklearn.load_model(model_uri)

In [15]:
model.predict(x_train)

array([0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 2,
       1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2,
       1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 2, 0, 2, 0, 0, 2, 1, 2, 2, 2, 2, 1,
       0, 0, 2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2,
       1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 2,
       1, 1, 2, 2, 0, 1, 2, 0, 1, 2])